In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import datetime
import time

from config import gkey

In [2]:
file = 'output_data/cities.csv'
weather_df = pd.read_csv(file)
weather_df.head()

,city,latitude,longitude,max temperature,humidity,cloudiness,wind speed,country,date
0,kirovskiy,45.0917,133.5136,73.78,94,100,3.65,RU,1659476212
1,tasiilaq,65.6145,-37.6368,48.36,60,18,4.25,GL,1659476214
2,waipawa,-41.4122,175.5153,54.52,78,71,17.16,NZ,1659476215
3,thompson,55.7435,-97.8558,57.36,94,100,6.91,CA,1659476217
4,poum,-20.2333,164.0167,74.97,88,72,15.50,NC,1659476218


### Humidity Heatmap

In [3]:
cities = weather_df[["latitude", "longitude"]]

humidity = weather_df['humidity']
max_humidity = humidity.max()
max_humidity

100

In [4]:
# Access maps with unique API key
gmaps.configure(api_key=gkey)

# Plot Heatmap
figure_layout = {
    'width': '700px',
    'height': '500px'
}

fig = gmaps.figure(layout=figure_layout,zoom_level = 1, center = (40.866667, 34.566667))

# Create heat layer
heat_layer = gmaps.heatmap_layer(cities, weights=humidity, 
                                 dissipating=False,max_intensity=max_humidity,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='500px', width='700px'))

### Vacation Spots

Criteria: 
* Temp between 65 and 80 degrees
* Wind speed below 4 mph
* 0% cloudiness

In [11]:
vacation_spots = weather_df.loc[
    ((weather_df["max temperature"]>=65) & (weather_df["max temperature"]<80)) &
    (weather_df['wind speed']<4)&
    (weather_df["cloudiness"]==0)
]
vacation_spots

,city,latitude,longitude,max temperature,humidity,cloudiness,wind speed,country,date
48,quchan,37.1060,58.5095,67.78,45,0,2.77,IR,1659476288
111,qixingtai,30.4135,111.8874,78.71,85,0,2.73,CN,1659476384
131,changji,44.0167,87.3167,77.79,38,0,2.24,CN,1659476413
237,paris,48.8534,2.3488,79.57,55,0,3.44,FR,1659476426
258,urumqi,43.8010,87.6005,74.35,38,0,2.24,CN,1659476605
361,vila,42.0304,-8.1588,67.69,75,0,0.96,PT,1659476782
402,cap malheureux,-19.9842,57.6142,66.47,75,0,1.99,MU,1659476847
468,pulandian,39.3953,121.9669,77.27,88,0,0.85,CN,1659476959
506,steinbach,50.1667,8.5667,75.38,53,0,2.30,DE,1659477020
537,neustadt,49.7117,12.0872,68.70,70,0,0.00,DE,1659477069


In [12]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotels = []
city = []
country = []
for index,row in vacation_spots.iterrows():
    my_params = {
    "location": f"{row['latitude']},{row['longitude']}",
    "radius": 5000,
    "keyword": "hotel",
    "key": gkey
    }
    response = requests.get(base_url,params=my_params).json()
    try:
        hotels.append(response['results'][0]['name'])
        city.append(row["city"])
        country.append(row["country"])
    except:
        # hotel probably not within specified distance
        print(f"No hotel within radius for {row['city']}")
    time.sleep(1)

In [13]:
hotels_df = pd.DataFrame({
    "hotel name":hotels,
    "city":city,
    "country":country
})
hotel_dict = hotels_df.to_dict('records')
full_df = pd.merge(hotels_df, weather_df, how = "left", on=["city","country"])
full_df

,hotel name,city,country,latitude,longitude,max temperature,humidity,cloudiness,wind speed,date
0,هتل اپارتمان سلطانی,quchan,IR,37.1060,58.5095,67.78,45,0,2.77,1659476288
1,Jing'an Hotel,qixingtai,CN,30.4135,111.8874,78.71,85,0,2.73,1659476384
2,Garden Hotel,changji,CN,44.0167,87.3167,77.79,38,0,2.24,1659476413
3,Hôtel Lutetia,paris,FR,48.8534,2.3488,79.57,55,0,3.44,1659476426
4,Sheraton Urumqi Hotel,urumqi,CN,43.8010,87.6005,74.35,38,0,2.24,1659476605
5,Hotel Castrum Villae,vila,PT,42.0304,-8.1588,67.69,75,0,0.96,1659476782
6,Coin de Mire Attitude Hotel,cap malheureux,MU,-19.9842,57.6142,66.47,75,0,1.99,1659476847
7,Xinghui Kairui Hotel,pulandian,CN,39.3953,121.9669,77.27,88,0,0.85,1659476959
8,Vienna House MQ Kronberg im Taunus,steinbach,DE,50.1667,8.5667,75.38,53,0,2.30,1659477020
9,Hotel Post,neustadt,DE,49.7117,12.0872,68.70,70,0,0.00,1659477069


In [14]:
cities = full_df[["latitude", "longitude"]]
fig = gmaps.figure(layout=figure_layout)

In [15]:
# many thanks to this documentation for helping me figure this part out!
# https://buildmedia.readthedocs.org/media/pdf/jupyter-gmaps/latest/jupyter-gmaps.pdf

info_box_template = """
<dl>
<dt>Hotel</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**hotel) for hotel in hotel_dict]

In [16]:
markers = gmaps.marker_layer(cities, info_box_content=hotel_info)                            

fig.add_layer(heat_layer)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='500px', width='700px'))